In [1]:
from model_implementation_lib import *

In [2]:
d_rf, d_mwra_base, d_dcr, d_logan, d_d_aberjona, d_h_alewife, l_locationtype, d_model_list = load_data(
	f_d_rf = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/rec_flag_2015_16.csv',
	f_d_mwra_base = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/mwra_base_recflag.csv',
	f_d_dcr = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/muni_dcr_hist.csv',
	f_d_logan = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/logan_hourly.csv',
	f_d_d_aberjona = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/01102500-aberjona-day.txt',
	f_d_h_alewife = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/01103025-alewife-hour.txt',
	f_l_locationtype = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/source_data/LocationTypeIDs.tsv',
	f_d_model_list = '/Users/jeff/Projects/myrwa/myrwa-rf-bact-prediction/RecFlag Model List.xlsx',
	)

Loading data


/Users/jeff/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (1,4,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
d_combined = pd.concat([d_rf, d_mwra_base, d_dcr])

In [4]:
col_cat = []

d_h_aberjona = hourly_flow_interp(d_d_aberjona, ['64138_00060_00003'])

flow_dfs = {'h_aberjona':(d_h_aberjona, '64138_00060_00003'), 'h_alewife':(d_h_alewife, '168619_00060')}
col_cat_dummies = prepare_data(
	db_dfs = {'comb':d_combined},
	flow_dfs = flow_dfs,
	l_locationtype = l_locationtype, col_cat = col_cat
	)

df_sim_flow_rainfall, hrs_sim = gen_flow_rainfall('2016-01-01 00:00:00', '2016-12-31 00:00:00', flow_dfs, d_logan['prcp_in'],)

Interpolating streamflow data
Preparing data


/Users/jeff/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Generating flow and rainfall data for simulation


In [5]:
standard_limits = {
	'Boating':{'ECOLI':1260,'ENT':350},
	'Swimming':{'ECOLI':235,'ENT':104},
	}

model_dic = {}

In [10]:
model_i = 0
model_spec = d_model_list.iloc[model_i]

result_db_f, IVs = engineer_features(
    result_db = d_combined, 
    CharacteristicIDs = [model_spec.CharacteristicID],
    Unit = 'MPN/100ml',
    flow_dfs = flow_dfs,
    col_cat_dummies = col_cat_dummies,
    d_logan = d_logan,
    )

sel = result_db_f.LocationID.isin(model_spec['LocationID(s)'].split( ' + '))
X_all, y_all, X_train, X_test, y_train, y_test, baseline = gen_train_test(
                                result_db_f[sel], 
                                {model_spec.CharacteristicID: standard_limits[model_spec.Standard][model_spec.CharacteristicID]}, 
                                IVs)

## Save model
model_name = '_'.join(model_spec.values)

Engineering features
Splitting train and test data


In [12]:
X_all

array([[  3.38708333e+01,  -3.04166667e-01,   1.57000000e+01, ...,
          0.00000000e+00,   5.00000000e-02,   4.00000000e-02],
       [  2.35125000e+01,  -1.12500000e-01,   1.02000000e+01, ...,
          2.70000000e-01,   0.00000000e+00,   0.00000000e+00],
       [  3.81333333e+01,   4.29166667e-01,   1.16000000e+01, ...,
          1.20000000e-01,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  5.66833333e+00,  -3.83333333e-02,   2.95000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.38250000e+01,  -1.29583333e+00,   2.44000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.56250000e+01,  -7.91666667e-02,   7.32000000e+00, ...,
          9.00000000e-02,   4.80000000e-01,   5.00000000e-02]])

In [ ]:
model_dic[model_name] = bacteria_model(
    model_spec, 
    X_all, y_all, X_train, X_test, y_train, y_test, 
    IVs, model_spec.Standard, model_spec.CharacteristicID, 'MPN/100ml',
    sensitivity = 0.5, locations = model_spec['LocationID(s)'].split(' + '))